In [1]:
from lxml import etree
import FeatureExtraction as FE
import numpy

In [2]:
sexual_predator_ids_file = '../../dataset/test/pan12-sexual-predator-identification-groundtruth-problem1.txt'

In [3]:
def create_csv(input_file_path, output_file_name, batch_size):
    tree = etree.parse(input_file_path)
    author_conversation_node_dictionary = FE.extract_author_conversation_node_dictionary_from_XML(tree)
    del tree
    
    output_file_csv = open(output_file_name, 'w+')
    output_string_list = ['autor', 'number of conversation', 'percent of conversations started by the author',
                         'difference between two preceding lines[s]', 'number of messages sent',
                         'average percent of lines in conversation', 'average percent of characters in conversation',
                         'number of characters sent by the author', 'mean time of messages sent',
                         'is sexual predator']
    output_string = ','.join(output_string_list) + "\n"
    
    sexual_predator_ids_list = FE.sexual_predator_ids(sexual_predator_ids_file)
    
    i = 0
    for author, conversation_nodes in author_conversation_node_dictionary.iteritems():
        output_list = [author,
                       len(conversation_nodes),
                       FE.average_trough_all_conversations(author, conversation_nodes, FE.is_starting_conversation),
                       FE.average_trough_all_conversations(author, conversation_nodes, 
                                    FE.avg_time_between_message_lines_in_seconds_for_author_in_conversation),
                       FE.number_of_messages_sent_by_the_author(author, conversation_nodes),
                       FE.average_trough_all_conversations(author, conversation_nodes,
                                                           FE.percentage_of_lines_in_conversation),
                       FE.average_trough_all_conversations(author, conversation_nodes,
                                                           FE.percentage_of_characters_in_conversation),
                       FE.number_of_characters_sent_by_the_author(author, conversation_nodes),
                       FE.mean_time_of_messages_sent(author, conversation_nodes),
                       '1' if author in sexual_predator_ids_list else '0'
                      ]
        output_string += ','.join(map(str, output_list)) + '\n'
        if i == batch_size:
            output_file_csv.write(output_string)
            output_string = ''
            i = -1
            
        i += 1
        
    del output_string
    del author_conversation_node_dictionary
    output_file_csv.close()

In [4]:
file_path='../../dataset/test/pan12-sexual-predator-identification-test-corpus-2012-05-17.xml'

In [5]:
create_csv(file_path, '../../csv/chat_based_features_test.csv', 10000)